# 💰 Personal Finance and Investment Advisor Agent
This notebook builds a specialized AI agent using the Agent Development Kit (ADK) that acts as a Personal Finance and Investment Advisor.

## ⚙️ Setup and Prerequisites

#### 1. Configure Your Gemini API Key (Error-Proof Setup)
Since the kaggle_secrets module caused an error, this section uses a more robust method to load your Gemini API Key, prioritizing the environment variable and falling back to a secure prompt if needed.

In [1]:
# In your local environment
!pip install -q google-adk

In [2]:
import os
import getpass
import uuid
import warnings
import asyncio
# Note: asyncio is necessary for running the `await test_agent_query()` calls
# You may need to run 'import asyncio' in a separate cell before the tests
# if your environment requires it.

# --- ADK Imports ---
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
warnings.filterwarnings("ignore")

# 1. Configure Your Gemini API Key (Error-Proof Setup)
# ----------------------------------------------------

# 1. Check for the API Key in the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    # 2. If not found, prompt the user to enter the key securely
    print("🔑 GOOGLE_API_KEY environment variable not found.")
    try:
        # Prompt securely using getpass (won't display the key)
        api_key_input = getpass.getpass("Please enter your Gemini API Key: ")
        os.environ["GOOGLE_API_KEY"] = api_key_input
        GOOGLE_API_KEY = api_key_input
        print("Key set successfully.")
    except Exception as e:
        print(f"Error reading API Key: {e}")

if GOOGLE_API_KEY:
    print("✅ Setup and authentication complete.")
else:
    # Raise an error if authentication failed to prevent subsequent errors
    raise ValueError("❌ Authentication failed. Please obtain and set your GOOGLE_API_KEY.")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


🔑 GOOGLE_API_KEY environment variable not found.
Please enter your Gemini API Key: ··········
Key set successfully.
✅ Setup and authentication complete.


#### 2. Configure Retry Options
Define the retry configuration to handle transient network errors when communicating with the LLM, as seen in the ADK notebooks.

In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry configuration set.")

✅ Retry configuration set.


## 🛠️ Section 1: Define Tools (Financial Data Service)
This agent uses a mock function to simulate accessing a financial data API, which the LLM can call like a tool.

In [4]:
def get_investment_info(investment_name: str) -> str:
    """
    Returns mock financial information for a given stock or fund.
    """
    financial_data = {
        "s&p 500": "S&P 500 Index: +12.5% YTD, Volatility: Low, Recommendation: Hold for long-term growth.",
        "nasdaq": "NASDAQ Composite: +18.2% YTD, Volatility: Medium, Recommendation: Growth sector focus, suitable for moderate risk.",
        "aapl": "Apple Inc. (AAPL): Current Price: $185.50, Last 30 Days: +3.1%, Dividend Yield: 0.5%, Analyst Consensus: Buy.",
        "tsla": "Tesla Inc. (TSLA): Current Price: $220.75, Last 30 Days: -5.9%, Volatility: High, Analyst Consensus: Hold.",
        "emergency fund": "Emergency Fund: Should cover 3-6 months of living expenses. Recommended: High-yield savings account or money market fund. (Not an investment, but a foundational financial pillar.)",
    }

    investment_lower = investment_name.lower().strip()

    if investment_lower in financial_data:
        return f"Investment Info: {financial_data[investment_lower]}"
    else:
        available = ", ".join([p.title() for p in financial_data.keys()])
        return f"Sorry, I don't have real-time data for '{investment_name}'. Available mock queries: {available}"

print("✅ Investment Info tool created.")

✅ Investment Info tool created.


## 🤖 Section 2: Create the Agent (Error Fixed)
We define the FinancialAdvisorAgent, ensuring the model name printing is robust by using a separate variable, which fixes the 'Gemini' object has no attribute 'name' and 'Gemini' object has no attribute 'model_name' errors.

In [5]:
MODEL_ID = "gemini-2.5-flash-lite"

# Create the agent
financial_advisor_agent = LlmAgent(
    # Use the stored model identifier
    model=Gemini(model=MODEL_ID, retry_options=retry_config),
    name="personal_finance_advisor",
    description="A knowledgeable agent providing general personal finance, investment, and market advice.",
    instruction="""
    You are a friendly and professional Personal Finance and Investment Advisor.
    Your primary goal is to provide accurate, balanced, and helpful financial guidance.

    When asked about specific investments or financial products:
    1. Use the `get_investment_info` tool to fetch current, relevant data.
    2. Analyze the tool's output to provide a clear summary, including performance, risk, and any recommendation.
    3. Always include a disclaimer: "Please remember this is general advice and not personalized financial recommendation."

    When asked about general finance topics (e.g., budgeting, debt), use your internal knowledge.
    Be concise, ethical, and always maintain a professional tone.
    """,
    tools=[get_investment_info],  # Register the financial tool
)

print("✅ Financial Advisor Agent created and configured.")
# FIX: Print the model identifier using the reliable MODEL_ID variable
print(f"   Model: {MODEL_ID}")
print(f"   Tool: {get_investment_info.__name__}()")

✅ Financial Advisor Agent created and configured.
   Model: gemini-2.5-flash-lite
   Tool: get_investment_info()


## 🧪 Section 3: Test Agent Functionality
This section defines and runs the test function, creating a new session for each query to simulate user interaction, as shown in the ADK examples.

Define test_agent_query Function

In [6]:
async def test_agent_query(user_query: str):
    """
    Tests the agent by creating a new session, running a query, and printing the response.
    """
    # Setup session management
    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "finance_app"
    user_id = "test_investor"
    session_id = f"advisor_session_{uuid.uuid4().hex[:8]}"

    # Create session
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the agent
    runner = Runner(
        agent=financial_advisor_agent, app_name=app_name, session_service=session_service
    )

    # Create the user message
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 Client: {user_query}")
    print(f"\n🤖 Advisor Response:")
    print("=" * 70)

    # Run the agent asynchronously
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("=" * 70)

Run Test Queries

In [7]:
print("🧪 Testing Financial Advisor Agent...\n")

# Note: You may need to wrap these calls in 'await' if running interactively.
# Example: await test_agent_query(...)

# Test 1: Investment Inquiry (Uses the tool)
await test_agent_query(
    "What is the current performance and recommendation for Apple (AAPL) stock?"
)

# Test 2: General Finance Inquiry (Uses internal knowledge)
await test_agent_query(
    "What is the first step I should take when starting a personal budget?"
)

# Test 3: Financial Concept Inquiry (Uses the tool for structured info)
await test_agent_query(
    "Can you explain the purpose of an emergency fund and what I should put it in?"
)

🧪 Testing Financial Advisor Agent...


👤 Client: What is the current performance and recommendation for Apple (AAPL) stock?

🤖 Advisor Response:



👤 Client: What is the first step I should take when starting a personal budget?

🤖 Advisor Response:
The first step to starting a personal budget is to track your income and expenses. This will give you a clear picture of where your money is coming from and where it's going. Once you have this data, you can start to categorize your spending and identify areas where you can save.

👤 Client: Can you explain the purpose of an emergency fund and what I should put it in?

🤖 Advisor Response:
An emergency fund is a crucial part of personal finance. Its main purpose is to cover unexpected expenses, such as job loss, medical emergencies, or urgent home repairs, without derailing your long-term financial goals or forcing you into debt.

**What to put it in:**

The key is to keep your emergency fund **safe, liquid, and easily accessible**. This means you'll want to avoid investments that carry a high risk of losing value or are difficult to withdraw from quickly. Here are some common options:



##  🧹 Section 4: Cleanup Session (Optional for Local Run)
 The notebooks you provided emphasized cleanup for two scenarios:

1. A2A Background Server: Stopping a server process started with subprocess.Popen.

2. Agent Engine Deployment: Deleting a deployed resource to avoid charges.

 Since this agent is currently running locally within your current Python session via the runner.run_async function, there is no background server process or cloud resource to delete. The session is cleaned up automatically when the cell finishes executing.

However, if this agent were part of an A2A system where a server was started, the cleanup would look like this:

In [8]:
print("\n🧹 Initiating Cleanup...")

# This agent runs in the current process and does not start a background server (like A2A)
# or deploy a cloud resource (like Agent Engine). Therefore, no code is required
# to explicitly stop a service or delete a resource.
# The agent instances and sessions are released automatically upon cell completion.

if 'financial_advisor_agent' in globals():
    del financial_advisor_agent
    print("✅ Local agent object deleted.")

if 'runner' in globals():
    del runner
    print("✅ Runner object deleted.")

print("ℹ️ Cleanup complete. No background processes or cloud resources were running.")


🧹 Initiating Cleanup...
✅ Local agent object deleted.
ℹ️ Cleanup complete. No background processes or cloud resources were running.
